In [ ]:
# import the raw data file
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/nypl_1850_51_split_v3.txt"
with open(file_path) as f:
  data = f.readlines()

Data Cleaning

In [ ]:
data[0] = 'A.'
import re

d = {}
value = []
i = 0
for line in data:
  line = line.strip()
  if line:
    if len(line) < 3: 
      # 1850 data is not split by '***' instead it is split by 'B., C., D.'
      if bool(re.match("([A-Z]+\.)", line)) == True:
        # save previous value
        if value:
          if value[0] == "'":
            value = value[1:]
          d[key] = value
        
        key = line
        value = []
    else:
      value.append(line)

if the first letter of the sentence is a special character remove it

In [ ]:

for key,value in d.items():
  n = len(value)
  for i in range(n):
    if (value[i][0] in ("'","*",",","^","!","_","-"," ")):
      value[i] = value[i][1:]



combine the two sentences if:

first 2 letters of the next sentence are:  ["B’", "E.", "W.","h.","C.","Av."]

If the next sentence starts with a different letter than its prior or next sentence

if the next sentence contains only one word

In [ ]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0

  while i < n:
    if i < n-2:
      if ((value[i+1][0:2] in ["B’", "E.", "W.","h.","C."]) or (value[i+1][0:2] in [". "]) or (value[i+1][0:4] == "Av. ")) :
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2

      elif ((value[i+1][0] != value[i][0]) and (value[i][0] == value[i+2][0]) and (value[i+1][0] != initial)) :
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2

      elif ((value[i+1][0] != initial) and (len(value[i+1].split(" ")) == 1)) :
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      
      elif ((value[i][-1] == "-") and (value[i+1][0] != initial)):
        s = value[i][:-1] + value[i+1]
        new_value.append(s)
        i += 2
            
      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value


Join the two sentences if the next sentence start with 'broadway', 'jersey', 'brooklyn'

and if the next sentence has only one word with length more than 4 (as most of the identifiers are single words with length 3)

remove "(see ", "-CONTINUED", "FOR INSERTION" keywords

In [ ]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0

  while i < n:
    if i < n-2:
      if (("Broadway" in value[i+1].split(" ")[0]) or ("Brooklyn" in value[i+1].split(" ")[0]) or (("Jersey" in value[i+1].split(" ")[0]) and ("John" not in value[i+1].split(" ")[1])) ):
        s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2

      elif (len(value[i+1].split(" ")) == 1) and (len(value[i+1].split(" ")[0]) > 4):
        s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2

      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value

  

It seems there are instances where 3 sentences make up an entry. So combine the three sentences ( identified as all start with different letters and the third sentence doesnot start with the same letter as the fourth sentence)

In [ ]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0

  while i < n:
    if i < n-2:
      if ((value[i+1][0] != value[i][0]) and (value[i][0] == value[i+2][0]) and (value[i+1][0] != initial)) :
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2

      elif ((value[i+1][0] != initial) and (len(value[i+1].split(" ")) == 1)) :
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      
      elif ((value[i+1][0] != value[i][0]) and (value[i][0] != value[i+2][0]) and (value[i+2][0] != value[i+3][0])) :
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1] + ' ' + value[i+2]
        else:
          s = value[i] + ' ' + value[i+1] + ' ' + value[i+2]
        new_value.append(s)
        i += 3
      

      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value


In [ ]:
#remove \"(see adv....\"\n",
for key,value in d.items():
    n = len(value)
    for i in range(n):
      ls = value[i].split()
      n2 = len(ls)
      for j in range(n2):
        if (("(see" in ls[j]) or ("(See" in ls[j])):
          value[i] = ' '.join(ls[:j])
          break

Identify business or person records

In [ ]:
# add an index to each record
# in another dictionary, for each index specify whether it's business or person record
index = 1
d_bp = []
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    if ls:
      if (ls[0][0] == "*" ):
        value[i] = value[i][1:]
        d_bp.append([str(index), "b"])
      elif (len(ls[0]) > 1 and ls[0].isupper()):
        d_bp.append([str(index), "b"])
      else:
        d_bp.append([str(index), "p"])
    
      value[i] = str(index) + "\t" + value[i]
      index += 1

In [ ]:
file = open("/content/nypl_1850_mn_clean_records.txt", "w")
for key,value in d.items():
  file.write(key + '\n')
  for record in value:
    file.write(record + '\n')
  file.write('\n')

file.close()

In [ ]:
file2 = open("/content/index_business_person.txt", "w")
for item in d_bp:
  file2.write(item[0] + ' ' + item[1] + '\n')
  
file2.close()

import the old cleaned file to compare

In [ ]:
# import the old cleaned file to compare
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/nypl_1850_51_starred_clean_old.txt"
with open(file_path) as f:
  data_clean = f.readlines()


import re
n = len(data_clean)
full_entry = []
for i in range(n):
  ls = data_clean[i].split()
  if ls:
    if bool(re.match("([A-Z]+\.)", ls[0])) != True:
      full_entry.append(' '.join(ls[1:]))

import the new cleaned file

In [ ]:
import re
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/nypl_1850_mn_clean_records.txt"
with open(file_path) as f:
  new_output = f.readlines()

n = len(new_output)
new_output_check = []
for i in range(n):
  ls = new_output[i].split()
  if ls:
    if bool(re.match("([A-Z]+\.)", ls[0])) != True:
      new_output_check.append(' '.join(ls[1:]))

check how many entries do not match with the old cleaned file

In [ ]:
left_new_entries = []

for i in new_output_check:
  if i in full_entry:
    continue
  else:
    left_new_entries.append(i)


print(len(new_output_check)) # records in new file
print(len(full_entry))      # records in old file
print(len(left_new_entries))  # extra records

79534
77695
5829


In [ ]:
left_new_entries

['Auchincloss & Croxon, com. mers. 192 Fulton',
 'Bagley William, clerk, bank, 50 Wall, h. 91 Henry, Brooklyn',
 'BAIN ALEXANDER, telegraph inventor, 181 Broadway, h. Irving house',
 'Baldwin Ezekiel, shoemaker, 79 Av. B',
 'Ball Isaac, pedler, 66 Mangin',
 'Bannon Michael, cooper, 304 Ninth',
 'Bannore Christopher, shoes, E. 13th n. Av. A',
 'Barratt Thomas, carman, 605 Fourth',
 'Barrowclough James, E. 24th c. Av. 3',
 'Barry James, marblepolisher, 294 Eleventh',
 'Barry John, carpenter, 60 Av. C',
 'Bates John C. sarsaparilla, 82 Nassau, h. Brooklyn',
 'Beach John T. paper, 43 Fulton, h. 22 Vesey',
 'Bear Isaac, drygoods, 105 Av. D, h. 105 Av. D',
 'Beard William, carpenter, 89 E. 11th',
 'Becom John J. sawyer, Av. C n. Eleventh',
 'Beebe Joseph D. rigger, 350 Tenth',
 'Beebe & Co. builders. W. 24th n. Av. 10',
 'Beers William P. butcher, 69 Orange, h. 77 Third',
 'Bender George W. tailor, 27 Av. A',
 'Bennett Cornelius, carman, 400 Tenth',
 'Bennett George, shoemaker, 131 Av. C',
 

there are some 5800 different records then the previous file

In [ ]:
file = open("/content/1850_mn_diff_new_entry.txt", "w")
for record in left_new_entries:
  file.write(record + '\n')
  file.write('\n')

file.close()

check what kind of records present in old file which are not in the newly cleaned file

In [ ]:
left_old_entries = []

for i in full_entry:
  if i in new_output_check:
    continue
  else:
    left_old_entries.append(i)


print(len(new_output_check)) # records in new file
print(len(full_entry))      # records in old file
print(len(left_old_entries))  # old records not present in new file

79534
77695
3992


In [ ]:
left_old_entries

['Auchincloss & Croxon, com. mers. 192 Fulton Bagley William, clerk, bank, 50 Wall, h. 91 Henry,',
 'Brooklyn BAIN ALEXANDER, telegraph inventor, 181',
 'Broadway, h. Irving house Baldwin Ezekiel, shoemaker, 79 Av. B',
 'Ball Isaac, pedler, 66 Mangin Bannon Michael, cooper, 304 Ninth',
 'Bannore Christopher, shoes, E. 13th n. Av. A Barratt Thomas, carman, 605 Fourth',
 'Barrowclough James, E. 24th c. Av. 3 Barry James, marblepolisher, 294 Eleventh',
 'Barry John, carpenter, 60 Av. C Bates John C. sarsaparilla, 82 Nassau, h. Brooklyn',
 'Beach John T. paper, 43 Fulton, h. 22 Vesey Bear Isaac, drygoods, 105 Av. D, h. 105 Av. D',
 'Beard William, carpenter, 89 E. 11th Becom John J. sawyer, Av. C n. Eleventh',
 'Beebe Joseph D. rigger, 350 Tenth Beebe & Co. builders. W. 24th n. Av. 10',
 'Beers William P. butcher, 69 Orange, h. 77 Third Bender George W. tailor, 27 Av. A',
 'Bennett Cornelius, carman, 400 Tenth Bennett George, shoemaker, 131 Av. C',
 'Bennett William, carman, 245 E. 13th Be